# Export tabular model to ONNX format

In [1]:
import numpy as np
import onnxruntime as rt
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier, XGBRegressor, DMatrix, train as train_xgb
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, to_onnx, update_registered_converter

from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost
from onnxmltools.convert import convert_xgboost as convert_xgboost_booster
import joblib
import pickle

## Preprocessors

copied from [Tabular_Model_F_Adam.ipynb](../Tabular_Model_F_Adam.ipynb)

In [2]:
# Define data type columns
ordinal_cols = ['pain', 'acuity']
ratio_cols   = ['temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp']

# Preprocess parameters
impOrd   = SimpleImputer(strategy='constant', fill_value=-1)
impRatio = SimpleImputer(strategy='mean')
scale    = RobustScaler(with_centering=False)
encode   = OrdinalEncoder()

# Simple imputing Preprocess
ord_pp_steps  = Pipeline([('missing',impOrd),('Ordinal',encode),('Scale',scale)])
ratio_pp_steps= Pipeline([('mean',impRatio),('Scale',scale)])

# create the preprocessor stage of final pipeline
t=[("ordinal",ord_pp_steps,ordinal_cols),('ratio',ratio_pp_steps ,ratio_cols)]
preprocessor = ColumnTransformer(transformers = t)

## Load XGBoost model

In [3]:
file_name = '../Saved_Models/xgb_tabular_model_4-1-24.pkl'

with open(file_name, 'rb') as file:
    loaded_xgb = pickle.load(file)

In [4]:
loaded_xgb.get_params()

{'objective': 'multi:softmax',
 'base_score': 0.5,
 'booster': 'gbtree',
 'callbacks': None,
 'colsample_bylevel': 0.0,
 'colsample_bynode': 0.0,
 'colsample_bytree': 0.25,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': 0.0,
 'grow_policy': None,
 'importance_type': 'gain',
 'interaction_constraints': None,
 'learning_rate': 0.05,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': 0.0,
 'max_depth': 1,
 'max_leaves': None,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 320,
 'n_jobs': -1,
 'num_parallel_tree': 1,
 'random_state': 711,
 'reg_alpha': 0.5,
 'reg_lambda': 0,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': 0.5,
 'tree_method': 'hist',
 'validate_parameters': False,
 'verbosity': 2,
 'num_class': 5,
 'gpu_id': None}

In [5]:
pipe = Pipeline([('preprocess',preprocessor), ('estimator', loaded_xgb)])

In [6]:
pipe.get_params()

{'memory': None,
 'steps': [('preprocess',
   ColumnTransformer(transformers=[('ordinal',
                                    Pipeline(steps=[('missing',
                                                     SimpleImputer(fill_value=-1,
                                                                   strategy='constant')),
                                                    ('Ordinal', OrdinalEncoder()),
                                                    ('Scale',
                                                     RobustScaler(with_centering=False))]),
                                    ['pain', 'acuity']),
                                   ('ratio',
                                    Pipeline(steps=[('mean', SimpleImputer()),
                                                    ('Scale',
                                                     RobustScaler(with_centering=False))]),
                                    ['temperature', 'heartrate', 'resprate',
                         

In [7]:
sample = pd.DataFrame([{'pain': 0, 'acuity': 2, 'temperature': 98.1, 'heartrate': 61, 'resprate': 18, 'o2sat': 95, 'sbp': 154, 'dbp': 85}], index=None)
sample

,pain,acuity,temperature,heartrate,resprate,o2sat,sbp,dbp
0,0,2,98.1,61,18,95,154,85


In [8]:
pipe.predict_proba(sample)

NotFittedError: This ColumnTransformer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.